<img src="files/iWAF.png">


Demo Setup -> done before the Demo.

* Setup VS
* Whitelist
* Signatures
* Learning preparation

-> Have a working Environment with some generated data for showing of analytics.

Demo Run ->

* Enable Learning
* Run Spider as Traffic generator
* Show results.

Teardown -> Removes Demo Environment

* In case anything went wrong to start from scratch.

# Setup

In [13]:
Controller_Version = "18.2.6"
### Import Demo Environment Variables (choose from demo_env_kneitinger, demo_env_vmware_us, demo_env_diagonalley)
from jupyter_demo_envs import demo_env_wbh as demo_env

#Set special items like a custom controller.

### Import python packages to setup Demo environment.
from jupyter_demo_imports import *

### Test Connection to Controller
#The api object will serve as the interface to the Controller.

api = check_setup(demo_env)
hackazon_vip = demo_env['vses']['Hackazon']['vip_ip']

Successful. Session ID:7c26cedzaojp67aimvg3v8bp3rufaes8


In [14]:
#Setup
tb = testbed_setup(demo_env)

#Whitelist add
add_whitelist_examples(demo_env, headers = demo_env['headers'], proxies = demo_env['proxies'])

#Whitelist example for log showcase
url = f'https://{hackazon_vip}/css/bootstrap.css'
send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

#Whitelist example for log showcase
payload = "contact_name=test&contact_email=test@example.com&contact_phone=555&contact_message=%3Cscript%3Ealert%28%22Alert%22%29%3C/script%3E&save=contact"
url = f'https://{hackazon_vip}/contact'    
send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

# generate normal and attack traffic
run_zap_spider(target="http://" + hackazon_vip, proxy=demo_env['zap_proxy'], loop=1)
run_zap_attack(target="http://" + hackazon_vip, proxy=demo_env['zap_proxy'], loop=1)
run_zap_spider(target="http://" + hackazon_vip, proxy=demo_env['zap_proxy'], loop=1)

print("\n----- Done. All automated data generated. -----")

change_policy_mode(api, demo_env, 'Hackazon', 'WAF_MODE_ENFORCEMENT')
#enable_learning_group(api, demo_env, "Hackazon", False)

payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

logs = None
while not logs:
    logs = print_log_entry(api, demo_env, "Hackazon", request_id)

scope = "rule"
logline = logs['results'][0]
create_exclusions(api, demo_env, "Hackazon", logline, scope)

payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

logs = None
while not logs:
    logs = print_log_entry(api, demo_env, "Hackazon", request_id)

#Choose scope for exclusion creation.
scope = "group"
#scope = "rule"
logline = logs['results'][0]
create_exclusions(api, demo_env, "Hackazon", logline, scope)

payload = "userEmail=goaway%40mailinator.com&userQuestion=What+about+ordering+Beer%3F+Hey%3B+%3A-%3E+Is+there+a+table+where+I+can%2C+say%3B+select+from+*+all+*+my+favorite+beers%3F+&_csrf_faq=9bFfqUaud0SuzsKDt0z2Canq7J7bEC74"
url = f'https://{hackazon_vip}/faq'    
request_id = send_post(url, payload, headers = demo_env['headers'], proxies = demo_env['proxies'])

print("\n----- All exclusion data generated. -----")


Data for Testbed Weissbrauhaus
Controller IP: 10.151.20.86
DVWA VIP: 10.151.80.87
Hackazon VIP: 10.151.80.88
Setting up Demo VS:DVWA
- Create Pool <Response [201]>
- Create new WAF profile <Response [201]>
- Create new WAF Policy <Response [201]>
- Create new Error Page Profile <Response [201]>
- Create new Response Data Script <Response [201]>
- Create VS with new WAF Policy <Response [201]>
Setting up Demo VS:Hackazon
- Create Pool <Response [201]>
- Create new WAF profile <Response [201]>
- Create new WAF Policy <Response [201]>
- Create new Error Page Profile <Response [201]>
- Create new Response Data Script <Response [201]>
- Create VS with new WAF Policy <Response [201]>
Setup done.
Data for Testbed Weissbrauhaus
Controller IP: 10.151.20.86
Adding whitelist entries to WAFPolicy:
<Response [200]>
Added Whitelist rules for css files, IPs and Detection mode for /contact
https://10.151.80.88/css/bootstrap.css
Request status code: 200
Fetching Request-ID
https://10.151.80.88/contact


# Demo

<img src="files/demo_overview.png" style="width:800px">

# Learning and Positive Security

<img src="files/overview_psm.png" style="width:800px" />

In [15]:
#Setup Learning
setup_learning(demo_env, headers = demo_env['headers'], proxies = demo_env['proxies'])
setup_positive_security_group(api, demo_env, 
                              headers = demo_env['headers'], 
                              proxies = demo_env['proxies'])
run_zap_spider(target="http://" + hackazon_vip, proxy=demo_env['zap_proxy'], loop=3)

# When done, query data:
print("\nWaiting for data generation...")
time.sleep(120)

# Add a denied logline:
print("\n----- Create rejected log -----")
url = f'https://{hackazon_vip}/product/view?id=16aasd'
logs = send_get(url, headers = demo_env['headers'], proxies = demo_env['proxies'])

print("\n----- Learning Data Overview -----")
print_learned_data(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Application Map Data -----")
print_application_map(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Promoted Rules -----")
print_auto_promoted_rules(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Done. -----")

# Finished. Rest happens in Avi UI

Data for Testbed Weissbrauhaus
Controller IP: 10.151.20.86
DVWA VIP: 10.151.80.87
Hackazon VIP: 10.151.80.88
Enable Learning on Policy - HackazonPolicy and Profile - HackazonProfile
Result: 200 OK
'PSMGroup Hackazon_Learning_Group_Demo has been set.'
Accessing target http://10.151.80.88
Spidering target http://10.151.80.88
Spider progress %: 97
Spider completed
Spidering target http://10.151.80.88
Spider progress %: 96
Spider completed
Spidering target http://10.151.80.88
Spider progress %: 97
Spider completed
All Spider runs completed
Waiting for data generation...
----- Create rejected log -----
https://10.151.80.88/product/view?id=16aasd
Request status code: 403
Fetching Request-ID
Request-ID of the request: g1S-Qv88-vbXe

----- Learning Data Overview -----
Current learning meta data:
{'duration': '0 hrs, 3 mins',
 'pct_high_confidence_params': 18.0,
 'total_params': 32,
 'total_requests': 1788,
 'total_uris': 456}

----- Application Map Data -----
/user/login
   Params found: 1
/wi

In [17]:
print("\n----- Learning Data Overview -----")
print_learned_data(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Application Map Data -----")
print_application_map(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Promoted Rules -----")
print_auto_promoted_rules(demo_env, api, headers = demo_env['headers'], proxies = demo_env['proxies'])
print("\n----- Done. -----")


----- Learning Data Overview -----
Current learning meta data:
{'duration': '0 hrs, 6 mins',
 'pct_high_confidence_params': 18.0,
 'total_params': 32,
 'total_requests': 1788,
 'total_uris': 456}

----- Application Map Data -----
/search
   Params found: 5
/css/
   Params found: 1
/faq
   Params found: 3
/css/nivo-themes/light/
   Params found: 1
/css/nivo-themes/
   Params found: 1
/product/view
   Params found: 1
/user/login
   Params found: 1
/wishlist
   Params found: 5
/wishlist/
   Params found: 1
/contact
   Params found: 5
/category/view
   Params found: 1
/css/nivo-themes/bar/
   Params found: 1
/user/register
   Params found: 6

----- Promoted Rules -----
/category/view
   id
    Pattern: ^[0-9]*$
    Length : 16
/product/view
   id
    Pattern: ^[0-9]*$
    Length : 16
/faq
   userEmail
    Pattern: ^[A-Za-z0-9 ._:,!?+*=@#\t-]*$
    Length : 64
   userQuestion
    Pattern: ^$
    Length : 0

----- Done. -----


# Teardown

In [ ]:
testbed_cleanup(demo_env)